In [4]:
import numpy as np, pandas as pd
import sklearn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/NLP_Project/

/content/drive/.shortcut-targets-by-id/1liGtjC9dkd3fXcYr3IVfYFk0iZ6z7rjm/NLP_Project


In [5]:
df = pd.read_csv("mpst_full_data.csv")
df = df.drop(['split'],axis=1)
df

,imdb_id,title,plot_synopsis,tags,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",imdb
...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,wikipedia


In [6]:
df.title[1]

'Dungeons & Dragons: The Book of Vile Darkness'

In [8]:
import re

In [14]:
!pip install contractions

     |████████████████████████████████| 266kB 8.2MB/s 
     |████████████████████████████████| 327kB 15.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85397 sha256=86ae6f7a039e2328bfb9f308ae3f49156202d42b47e3297d6ca7eb2e6acef9b5
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
import re
import contractions
from bs4 import BeautifulSoup
check=["one","two","three","four","five","six","seven","eight","nine","ten","iii","i","ii","iv","v","vi","vii","viii","ix","x"]
preprocessed_title=[]
for i in df['title'].values:
    i= re.sub(r"http\S+","", i)
    i= BeautifulSoup(i, 'lxml').get_text()
    i= re.sub("\S*\d\S*", "",i).strip()
    i= re.sub('[^A-Za-z]+', ' ', i)
    i= contractions.fix(i)
    i= ' '.join(j.lower() for j in i.split() if j.lower() not in stopwords)
    g=i.split()
    if("part" in g):
      ind=g.index("part")
      if((ind+1<len(g)-1) and (g[ind+1] in check)):
        g.remove(g[ind])
        g.remove(g[ind+1])
    i= ' '.join(g)
    preprocessed_title.append(i.strip())
df['preprocessed_title']=preprocessed_title

In [20]:
df

,imdb_id,title,plot_synopsis,tags,synopsis_source,preprocessed_title
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",imdb,tre volti della paura
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,imdb,dungeons dragons book vile darkness
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,imdb,shop around corner
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",imdb,mr holland opus
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",imdb,scarface
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",wikipedia,lucky numbers
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",wikipedia,iron man
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,wikipedia,play dirty
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,wikipedia,high wall


In [21]:
def remove_spaces(x):
    return x.replace(" ", "")
df['tags']=df['tags'].apply(remove_spaces)
df['tags']

0                    cult,horror,gothic,murder,atmospheric
1                                                 violence
2                                                 romantic
3                      inspiring,romantic,stupid,feel-good
4        cruelty,murder,dramatic,cult,violence,atmosphe...
                               ...                        
14823                                        comedy,murder
14824                              goodversusevil,violence
14825                                              antiwar
14826                                               murder
14827                                        christianfilm
Name: tags, Length: 14828, dtype: object

In [22]:
train, validate, test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.85*len(df))])

In [23]:
train=train.reset_index()
validate=validate.reset_index()
test=test.reset_index()

In [26]:
train

,index,imdb_id,title,plot_synopsis,tags,synopsis_source,preprocessed_title
0,8668,tt0102103,Impromptu,"Since getting divorced, Baroness Amantine-Luci...",violence,wikipedia,impromptu
1,10353,tt0431265,Metamorphosis,"=== Part I ===\nOne day, Gregor Samsa, a trave...",paranormal,wikipedia,metamorphosis
2,14814,tt3289956,The Autopsy of Jane Doe,An unidentified corpse of a woman is found hal...,"dark,cruelty,murder,paranormal,violence,sadist",wikipedia,autopsy jane doe
3,11741,tt2085930,Starship Troopers: Invasion,"On an asteroid, the Terran Federation's Fort C...","paranormal,violence",wikipedia,starship troopers invasion
4,8151,tt0441263,Fei hap siu baak lung,In a twist departing from the standard superhe...,romantic,wikipedia,fei hap siu baak lung
...,...,...,...,...,...,...,...
10374,5156,tt0112870,Dilwale Dulhania Le Jayenge,Chaudhry Baldev Singh (Amrish Puri) is settled...,romantic,imdb,dilwale dulhania le jayenge
10375,5813,tt0116282,Fargo,The movie opens with a car towing a new tan Ol...,"comedy,dark,neonoir,murder,boring,bleak,cult,v...",imdb,fargo
10376,7257,tt0316188,Raising Victor Vargas,Victor is a teenager growing up in the Lower E...,"romantic,humor,realism",wikipedia,raising victor vargas
10377,6107,tt0205700,Titus,A boy eating lunch in a 1950s-style kitchen pl...,flashback,wikipedia,titus


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['tags']).toarray()
y_test = vectorizer.transform(test['tags']).toarray()
print(y_train)

[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [28]:
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')


In [33]:
genres = set()
for t in df['tags'].str.split(','):
    genres |= set(t)


In [ ]:
cnt_word_given_class = defaultdict(lambda: defaultdict(lambda :0))
for i,row in train.iterrows():
    classes = train[target].unique()
    for word in tokenizer.tokenize(row['title']):
        cnt_word_given_class[word][row[target]]+=1.0
cnt_classes = {c:len(train[train[target]==c].index) for c in classes}
V = len(cnt_word_given_class.keys())